# 第5章 深度学习计算

## 5.1 层和块

### 练习 5.1.1 

如果将MySequential中存储块的方式更改为Python列表，会出现什么样的问题？ 

**解答：** 

### 练习 5.1.2

实现一个块，它以两个块为参数，例如net1和net2，并返回前向传播中两个网络的串联输出。这也被称为平行块。

**解答：**

### 练习 5.1.3

假设我们想要连接同一网络的多个实例。实现一个函数，该函数生成同一个块的多个实例，并在此基础上构建更大的网络。

**解答：** 

## 5.2 参数管理 

### 练习 5.2.1

使用`FancyMLP`模型，访问各个层的参数。

 **解答：**

### 练习 5.2.2

查看初始化模块文档以了解不同的初始化方法。

**解答：**

### 练习 5.2.3

构建包含共享参数层的多层感知机并对其进行训练。在训练过程中，观察模型各层的参数和梯度。

**解答：** 

### 练习 5.2.4

为什么共享参数是个好主意？

**解答：** 

## 5.3 延后初始化 

### 练习 5.3.1 

如果指定了第一层的输入尺寸，但没有指定后续层的尺寸，会发生什么？是否立即进行初始化？

**解答：** 

### 练习 5.3.2

如果指定了不匹配的维度会发生什么？

**解答：** 

### 练习 5.3.3 

如果输入具有不同的维度，需要做什么？提示：查看参数绑定的相关内容。

**解答：**

## 5.4 自定义层 

### 练习 5.4.1 

设计一个接受输入并计算张量降维的层，它返回$y_k = \sum_{i, j} W_{ijk} x_i x_j$。

**解答：** 

### 练习 5.4.2 

设计一个返回输入数据的傅立叶系数前半部分的层。

**解答：** 

## 5.5 读写文件 

###  练习 5.5.1

即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？

**解答：** 

### 练习 5.5.2

假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如想在一个新的网络中使用之前网络的前两层，该怎么做？

**解答：** 

### 练习 5.5.3

如何同时保存网络架构和参数？需要对架构加上什么限制？

**解答：** 

## 5.6 GPU 

### 练习 5.6.1 

尝试一个计算量更大的任务，比如大矩阵的乘法，看看CPU和GPU之间的速度差异。再试一个计算量很小的任务呢？

### 练习 5.6.2

我们应该如何在GPU上读写模型参数？

**解答：** 

### 练习 5.6.3 

测量计算1000个$100 \times 100$矩阵的矩阵乘法所需的时间，并记录输出矩阵的Frobenius范数，一次记录一个结果，而不是在GPU上保存日志并仅传输最终结果。

**解答:** 

### 练习 5.6.4

测量同时在两个GPU上执行两个矩阵乘法与在一个GPU上按顺序执行两个矩阵乘法所需的时间。提示：应该看到近乎线性的缩放。

**解答：** 